In [1]:
#!pip install rake_nltk

In [107]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

df = pd.read_csv('final-movie-database.csv')

df = df[['Title','Genre','Director','Actors','Plot']]
df=df[:51]

df=df.set_index("Title")
print(df.head())

                                                 Genre               Director  \
Title                                                                           
Ad Astra         Adventure, Drama, Mystery                         James Gray   
Hereditary          Drama, Horror, Mystery                          Ari Aster   
The Firm          Drama, Mystery, Thriller                     Sydney Pollack   
Ready or Not       Comedy, Horror, Mystery              Matt Bettinelli-Olpin   
Double Jeopardy      Crime, Drama, Mystery                    Bruce Beresford   

                                                            Actors  \
Title                                                                
Ad Astra         ['Brad Pitt', 'Tommy Lee Jones', 'Ruth Negga',...   
Hereditary       ['Toni Collette', 'Milly Shapiro', 'Gabriel By...   
The Firm         ['Tom Cruise', 'Jeanne Tripplehorn', 'Gene Hac...   
Ready or Not     ['Tyler Gillett', 'Samara Weaving', 'Adam Brod...   
Double Jeopa

In [108]:
# initializing the new column
df['Key_words'] = ""
#print(df["Actors"])

for index, row in df.iterrows():
    plot = row['Plot']
    
    # instantiating Rake, by default it uses english stopwords from NLTK
    # and discards all puntuation characters as well
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)

    # getting the dictionary whith key words as keys and their scores as values
    key_words_dict_scores = r.get_word_degrees()
    
    # assigning the key words to the new column for the corresponding movie
    row['Key_words'] = list(key_words_dict_scores.keys())

# dropping the Plot column
df.drop(columns = ['Plot'], inplace = True)

In [109]:
df['Actors'] = [word.lstrip('[') for word in df['Actors']]
df['Actors'] = [word.rstrip(']') for word in df['Actors']]
df["Actors"] = df["Actors"].str.replace(" ","")
df["Actors"] = df["Actors"].str.replace("\'","")
df["Actors"] = df["Actors"].str.replace(","," ")
df.Actors


Title
Ad Astra                                        BradPitt TommyLeeJones RuthNegga DonaldSutherland
Hereditary                                       ToniCollette MillyShapiro GabrielByrne AlexWolff
The Firm                                        TomCruise JeanneTripplehorn GeneHackman HalHol...
Ready or Not                                    TylerGillett SamaraWeaving AdamBrody "MarkOBri...
Double Jeopardy                                 AshleyJudd TommyLeeJones BruceGreenwood Benjam...
The Vast of Night                               SierraMcCormick JakeHorowitz GailCronauer Bruc...
A Simple Favor                                  AnnaKendrick BlakeLively HenryGolding AndrewRa...
The Lighthouse                                  RobertPattinson WillemDafoe ValeriiaKaraman Lo...
The Prestige                                    ChristianBale HughJackman ScarlettJohansson Mi...
The Hateful Eight                               SamuelL.Jackson KurtRussell JenniferJasonLeigh...
Watchmen      

In [110]:

df["Genre"] = df["Genre"].str.lower()
df['Genre'] = [word.rstrip() for word in df['Genre']]
df["Director"] = df["Director"].str.lower()
df["Actors"] = df["Actors"].str.lower()

#df["Genre"] = df["Genre"].str.replace(" ","")
df["Director"] = df["Director"].str.replace(" ","")
#df["Actors"] = df["Actors"].str.replace(" ","")
df["Genre"] = df["Genre"].str.replace(","," ")
#df["Director"] = df["Director"].str.replace(","," ")
#df["Actors"] = df["Actors"].str.replace(","," ")
print(df["Actors"])
df["bag_of_words"]=df["Genre"]+" "+df["Director"]+" "+df["Actors"]+" "+str(df["Key_words"])
df["bag_of_words"] = df["bag_of_words"].str.replace(","," ")
df.head()

Title
Ad Astra                                        bradpitt tommyleejones ruthnegga donaldsutherland
Hereditary                                       tonicollette millyshapiro gabrielbyrne alexwolff
The Firm                                        tomcruise jeannetripplehorn genehackman halhol...
Ready or Not                                    tylergillett samaraweaving adambrody "markobri...
Double Jeopardy                                 ashleyjudd tommyleejones brucegreenwood benjam...
The Vast of Night                               sierramccormick jakehorowitz gailcronauer bruc...
A Simple Favor                                  annakendrick blakelively henrygolding andrewra...
The Lighthouse                                  robertpattinson willemdafoe valeriiakaraman lo...
The Prestige                                    christianbale hughjackman scarlettjohansson mi...
The Hateful Eight                               samuell.jackson kurtrussell jenniferjasonleigh...
Watchmen      

,Genre,Director,Actors,Key_words,bag_of_words
Title,,,,,
Ad Astra,adventure drama mystery,jamesgray,bradpitt tommyleejones ruthnegga donaldsutherland,"[unforgiving, solar, system, missing, father, ...",adventure drama mystery jamesgray bradpitt t...
Hereditary,drama horror mystery,ariaster,tonicollette millyshapiro gabrielbyrne alexwolff,"[grieving, family, haunted, disturbing, occurr...",drama horror mystery ariaster tonicollette m...
The Firm,drama mystery thriller,sydneypollack,tomcruise jeannetripplehorn genehackman halhol...,"[sinister, dark, side, young, lawyer, joins, p...",drama mystery thriller sydneypollack tomcrui...
Ready or Not,comedy horror mystery,mattbettinelli-olpin,"tylergillett samaraweaving adambrody ""markobri...","[bride, terrifying, game, sinister, turn, laws...",comedy horror mystery mattbettinelli-olpin t...
Double Jeopardy,crime drama mystery,bruceberesford,ashleyjudd tommyleejones brucegreenwood benjam...,"[already, still, alive, prosecuted, crime, mur...",crime drama mystery bruceberesford ashleyjud...


In [111]:
print(df["bag_of_words"])


Title
Ad Astra                                        adventure  drama  mystery jamesgray bradpitt t...
Hereditary                                      drama  horror  mystery ariaster tonicollette m...
The Firm                                        drama  mystery  thriller sydneypollack tomcrui...
Ready or Not                                    comedy  horror  mystery mattbettinelli-olpin t...
Double Jeopardy                                 crime  drama  mystery bruceberesford ashleyjud...
The Vast of Night                               drama  mystery  sci-fi andrewpatterson sierram...
A Simple Favor                                  comedy  crime  drama paulfeig annakendrick bla...
The Lighthouse                                  drama  fantasy  horror roberteggers robertpatt...
The Prestige                                    drama  mystery  sci-fi christophernolan christ...
The Hateful Eight                               crime  drama  mystery quentintarantino samuell...
Watchmen      

In [112]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)

In [113]:
print(cosine_sim)


[[1.         0.99414062 0.99414062 ... 0.99414062 0.9931712  0.99323337]
 [0.99414062 1.         0.99414062 ... 0.99414062 0.9931712  0.99226341]
 [0.99414062 0.99414062 1.         ... 0.99414062 0.9931712  0.99226341]
 ...
 [0.99414062 0.99414062 0.99414062 ... 1.         0.9931712  0.99226341]
 [0.9931712  0.9931712  0.9931712  ... 0.9931712  1.         0.99226483]
 [0.99323337 0.99226341 0.99226341 ... 0.99226341 0.99226483 1.        ]]


In [124]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use in the function to match the indexes
indices = pd.Series(df.index)
#print(indices)

#  defining the function that takes in movie title 
# as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    # initializing the empty list of recommended movies
    recommended_movies = []
    
    # gettin the index of the movie that matches the title
    idx = indices[indices == title].index[0]
    #print(idx)

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    #print(score_series)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:51].index)
    #print(top_10_indexes)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(indices)[i])
        #print(indices[i])
        #print(recommended_movies)
        #print(df.index)
    return recommended_movies

In [121]:
recommendations("Pearl Harbor")

IndexError: index 0 is out of bounds for axis 0 with size 0

In [122]:
recommendations("Atonement")

['Blade Runner 2049',
 'Se7en',
 'Prisoners',
 'Double Jeopardy',
 'The Night Clerk',
 'Watchmen',
 'Eyes Wide Shut',
 'Gone Girl',
 'Apocalypse Now',
 'Inheritance',
 'Ad Astra',
 'Hereditary',
 'The Firm',
 'The Hateful Eight',
 'The Vast of Night',
 'The Beach House',
 'The Prestige',
 'Ex Machina',
 'Shutter Island',
 'Cloud Atlas',
 'Murder on the Orient Express',
 'Gretel & Hansel',
 'Sleepy Hollow',
 'Hot Fuzz',
 'The Guest',
 'Red Riding Hood',
 'The Usual Suspects',
 'Psycho',
 'Scary Stories to Tell in the Dark',
 'Get Out',
 'Vivarium',
 'I See You',
 'Harry Potter and the Deathly Hallows: Part 2',
 'Us',
 'Memento',
 'A Simple Favor',
 'The Lighthouse',
 'Freaks',
 'The Thing',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Order of the Phoenix',
 'Sen to Chihiro no kamikakushi',
 'Winchester',
 'Ready or Not',
 'Fantasy Island',
 'Coco',
 'Game Night',
 'Zootopia']

In [123]:
recommendations("Gone Girl")

['Se7en',
 'Eyes Wide Shut',
 'The Firm',
 'Inheritance',
 'Shutter Island',
 'The Usual Suspects',
 'Blade Runner 2049',
 'Prisoners',
 'Double Jeopardy',
 'The Night Clerk',
 'Watchmen',
 'Apocalypse Now',
 'Get Out',
 'Us',
 'Ad Astra',
 'Atonement',
 'Memento',
 'Psycho',
 'Hereditary',
 'The Hateful Eight',
 'Ex Machina',
 'The Beach House',
 'The Prestige',
 'The Vast of Night',
 'Cloud Atlas',
 'Murder on the Orient Express',
 'Gretel & Hansel',
 'Red Riding Hood',
 'Sleepy Hollow',
 'The Guest',
 'Hot Fuzz',
 'Vivarium',
 'A Simple Favor',
 'Scary Stories to Tell in the Dark',
 'I See You',
 'Harry Potter and the Deathly Hallows: Part 2',
 'The Lighthouse',
 'Freaks',
 'The Thing',
 'Harry Potter and the Chamber of Secrets',
 'Harry Potter and the Goblet of Fire',
 'Harry Potter and the Prisoner of Azkaban',
 'Harry Potter and the Order of the Phoenix',
 'Sen to Chihiro no kamikakushi',
 'Winchester',
 'Ready or Not',
 'Fantasy Island',
 'Coco',
 'Game Night',
 'Zootopia']